<a href="https://colab.research.google.com/github/Mrajie88/Competition/blob/master/competition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
import lightgbm as lgb
import time
import torch
from mpl_toolkits.mplot3d import Axes3D
import os

In [0]:
print("cpu" if not torch.cuda.is_available() else torch.cuda.get_device_name())

Tesla P4


In [0]:
path = "/content/drive/My Drive/"
train_df_source = pd.read_csv(path+'train.csv')
test_df_source = pd.read_csv(path+'test.csv')
event_df_source = pd.read_csv(path+'event.csv')

In [0]:
def f_eng(df, event_df, is_train=True):
    df = pd.merge(df, event_df.loc[:, ['event_id', 'energymc', 'thetamc', 'phimc', 'xcmc', 'ycmc']], how='left',
                  on='event_id')
    # df['dis'] = np.sqrt(df['x']**2+df['y']**2+df['t']**2)
    # 时间差
    df['t_o']=df['t']-df['terror']

    df['q_mean'] = df['q']-df['q'].groupby(df['event_id']).transform(np.mean)
    df['t_mean'] = df['t']-df['t'].groupby(df['event_id']).transform(np.mean)
    df['t_std'] = df["t"].groupby(df["event_id"]).transform(np.std)
    df['t_ptp'] = df["t"].groupby(df["event_id"]).transform(np.ptp)
    df["x_xcmc"] = df["x"]-df["xcmc"]
    df["y_ycmc"] = df["y"] - df["ycmc"]
    df["dis"] = np.sqrt(df["x_xcmc"] ** 2+df["y_ycmc"] ** 2 +df["t"] ** 2)
    
    if is_train:
        df = df.drop('flag', axis=1)
    df = df.drop(['hit_id', 'z', 'event_id'], axis=1)
    return df

In [0]:
labels = train_df_source['flag']
train_df = f_eng(train_df_source, event_df_source)
test_df = f_eng(test_df_source, event_df_source, False)
print(train_df.columns.values.tolist())
# 小样本训练
train_df = train_df.iloc[:10000000, :]
labels = labels[:10000000]

train_x, val_x, train_y, val_y = train_test_split(train_df, labels, test_size=0.33, random_state=42)

/usr/local/lib/python3.6/dist-packages/numpy/core/fromnumeric.py:2495: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


['x', 'y', 't', 'terror', 'q', 'energymc', 'thetamc', 'phimc', 'xcmc', 'ycmc', 't_o', 'q_mean', 't_mean', 't_std', 't_ptp', 'x_xcmc', 'y_ycmc', 'dis']


In [0]:
fea_imp_list = []
params = {
    "task":"train",
    "boosting_type":"gbdt",
    "metric":"auc",
    "num_leaves":255,
    "learning_rate":0.01,
    "feature_fraction":0.9,
    "bagging_fraction":0.8,
    "verbose":200
}

In [0]:
batch_size = 5000000
gbm = None
#with open("/content/drive/My Drive/lgb_model_v0.txt","r") as f:
#  gbm = lgb.Booster({'model_str':f.read()})
print("=========================================================模型读取成功======================================================================================")
max_len = 0
for i in range(1,len(train_x),batch_size):
  if i+ batch_size >len(train_x):
    max_len = batch_size
  else:
    max_len = len(train_x)-i-1
  print("=========================================================数据准备======================================================================================")
  lgb_train = lgb.Dataset(train_x.iloc[i:i+max_len,:],train_y.iloc[i:i+max_len])
  lgb_test = lgb.Dataset(val_x,val_y,reference = lgb_train)
  print("=========================================================模型训练======================================================================================")
  gbm = lgb.train(params,lgb_train,num_boost_round=10000,init_model = gbm,valid_sets = lgb_test,early_stopping_rounds = 200,keep_training_booster=True)
  gbm.save_model("/content/drive/My Drive/lgb_model_v"+str(i//batch_size)+".txt")
  print("============================================================模型训练完成===========================================================================================")
#fea_imp_list.append(gbm.feature_importances_)

流式输出内容被截断，无法输出最后 5000 行内容。
[7575]	valid_0's auc: 0.996128
[7576]	valid_0's auc: 0.996128
[7577]	valid_0's auc: 0.996128
[7578]	valid_0's auc: 0.996129
[7579]	valid_0's auc: 0.996129
[7580]	valid_0's auc: 0.996129
[7581]	valid_0's auc: 0.996129
[7582]	valid_0's auc: 0.996129
[7583]	valid_0's auc: 0.996129
[7584]	valid_0's auc: 0.996129
[7585]	valid_0's auc: 0.996129
[7586]	valid_0's auc: 0.996129
[7587]	valid_0's auc: 0.996129
[7588]	valid_0's auc: 0.996129
[7589]	valid_0's auc: 0.996129
[7590]	valid_0's auc: 0.996129
[7591]	valid_0's auc: 0.996129
[7592]	valid_0's auc: 0.996129
[7593]	valid_0's auc: 0.996129
[7594]	valid_0's auc: 0.996129
[7595]	valid_0's auc: 0.996129
[7596]	valid_0's auc: 0.996129
[7597]	valid_0's auc: 0.99613
[7598]	valid_0's auc: 0.99613
[7599]	valid_0's auc: 0.99613
[7600]	valid_0's auc: 0.99613
[7601]	valid_0's auc: 0.996131
[7602]	valid_0's auc: 0.996131
[7603]	valid_0's auc: 0.996131
[7604]	valid_0's auc: 0.99613
[7605]	valid_0's auc: 0.996131
[7606]	valid_0's 

AttributeError: ignored

In [0]:
gbm = None
with open("/content/drive/My Drive/lgb_model_v0.txt","r") as f:
  gbm = lgb.Booster({'model_str':f.read()})

In [0]:
pred = gbm.predict(test_df)

In [0]:
pred

array([0.00757676, 0.08355701, 0.00528702, ..., 0.01825989, 0.00594394,
       0.00323018])

In [0]:
t = time.time()
test_pre = pd.DataFrame(pred, columns=['flag_pred'])
print('test_pre runtime:', time.time() - t)
sub = pd.concat([test_df_source['hit_id'], test_pre,test_df_source['event_id']], axis=1)
#fea_imp_list.append(gbm.feature_importances_)
print('runtime:', time.time() - t)

test_pre runtime: 0.0015943050384521484
runtime: 0.043180227279663086


In [0]:
# 特征重要性
fea_imp_dict = dict(zip(train_df.columns.values, np.mean(fea_imp_list, axis=0)))
fea_imp_item = sorted(fea_imp_dict.items(), key=lambda x: x[1], reverse=True)
for f, imp in fea_imp_item:
    print('{} = {}'.format(f, imp))

/usr/local/lib/python3.6/dist-packages/numpy/core/fromnumeric.py:3257: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.6/dist-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


TypeError: ignored

In [0]:
# sub.to_csv('drive/My Drive/sub_prob_{}_{}_{}.csv'.format(best_auc, best_f1,sub['flag_pred'].mean()),index=False)
best_t = 0.7
sub['flag_pred'] = sub['flag_pred'].apply(lambda x: 1 if x >= best_t else 0)
sub.to_csv('drive/My Drive/submition_{}_{}.csv'.format(time.asctime( time.localtime(time.time()) ),sub['flag_pred'].mean()),index=False)

print('runtime:', time.time() - t)
print('finish.')

runtime: 765.3468747138977
finish.
